In [30]:
import numpy as np
import pandas as pd

In [35]:
soi = pd.read_csv("soi.txt", sep = "  " ,header=0, engine="python")
#sst = pd.read_csv("sst.txt", sep="  ", on_bad_lines="skip", header=0, index_col=False, dtype="str", engine="python")
olr = pd.read_fwf("olr.txt", skiprows=1, names = ["Year", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul" ,"Aug", "Sep", "Oct", "Nov", "Dec"], 
                    colspecs = [(0, 4), (4,  10), (10, 16), (16, 22), (22, 28), (28, 34), (34, 40), (40, 46), (46, 52), (52, 58), (58, 64), (64, 70), (70, 76)])

sst = pd.read_fwf("sst.txt", skiprows=1, names = ["Year", "Mon", "Nino1.2", "Anom1.2", "Nino3", "Anom3", "Nino 4", "Anom4", "Nino3.4", "Anom3.4"], 
                    colspecs = [(0, 4), (4,  11), (11, 19), (19, 27), (27, 35), (35, 43), (43, 51), (51, 59), (59, 67), (67, -1)])


In [36]:
sst["Date"] = sst["Mon"].astype(str) + " " + sst["Year"].astype(str)
sst["Date"] = pd.to_datetime(sst["Date"])
sst.sort_values("Date", inplace=True)
sst = sst.drop(["Year", "Mon"], axis=1)
sst = sst.set_index("Date")
sst.head()

,Nino1.2,Anom1.2,Nino3,Anom3,Nino 4,Anom4,Nino3.4,Anom3.4
Date,,,,,,,,
1982-01-01,24.28,-0.24,25.84,0.17,28.01,-0.21,26.65,0.08
1982-02-01,25.38,-0.72,26.26,-0.11,27.99,-0.11,26.54,-0.20
1982-03-01,25.22,-1.38,26.92,-0.25,28.18,-0.05,27.09,-0.14
1982-04-01,24.57,-1.16,27.52,-0.05,28.61,0.10,27.83,0.02
1982-05-01,24.00,-0.62,27.70,0.49,29.19,0.40,28.37,0.49


In [144]:
olr_melted = olr.melt(id_vars = "Year", var_name = "Month", value_name="OLR").sort_values("Year")
olr_melted["Date"] = olr_melted["Month"].astype(str) + " " + olr_melted["Year"].astype(str)
olr_melted.head()
olr_melted["Date"] = pd.to_datetime(olr_melted["Date"])
olr_melted.sort_values("Date", inplace=True)
olr_melted = olr_melted.drop(["Year", "Month"], axis=1)
olr_melted = olr_melted[["Date", "OLR"]]
olr_melted.head()

,Date,OLR
0,1974-01-01,-999.9
50,1974-02-01,-999.9
100,1974-03-01,-999.9
150,1974-04-01,-999.9
200,1974-05-01,-999.9


In [145]:
olr_melted = olr_melted.drop(olr_melted.head(60).index)
olr_melted = olr_melted.replace(-999.9, np.nan)
result = olr_melted.groupby([olr_melted.Date.dt.month]).agg({"OLR" : ["mean"]})
for i in range(6):
    mean = result.iloc[5+i, 0]
    olr_melted.iloc[365 + i, 1]= mean
olr_melted = olr_melted.set_index("Date")


,OLR
Date,
1979-01-01,-0.7
1979-02-01,-0.6
1979-03-01,-0.3
1979-04-01,0.3
1979-05-01,0.0
...,...
2023-08-01,NaN
2023-09-01,NaN
2023-10-01,NaN


In [146]:
soi_melted = soi.melt(id_vars = "YEAR", var_name = "Month", value_name="SOI").sort_values("YEAR")
soi_melted["Date"] = soi_melted["Month"].astype(str) + " " + soi_melted["YEAR"].astype(str)
soi_melted.head()
soi_melted["Date"] = pd.to_datetime(soi_melted["Date"])
soi_melted.sort_values("Date", inplace=True)
soi_melted = soi_melted.drop(["YEAR", "Month"], axis=1)
soi_melted = soi_melted[["Date", "SOI"]]
soi_melted = soi_melted.set_index("Date")
soi_melted.head()

,SOI
Date,
1951-01-01,1.5
1951-02-01,0.9
1951-03-01,-0.1
1951-04-01,-0.3
1951-05-01,-0.7


In [147]:
df = sst.merge(olr_melted, on=["Date"], how="outer")
df = df.merge(soi_melted, on=["Date"], how="outer")
df = df.sort_values("Date")

df.to_csv("MergedData.csv")

In [153]:
df1 = df["SOI"]                         #Dataframe que solo incluye SOI, empieza en 1954 y termina en abril 2023
df1 = df1.drop(df1.tail(8).index)
df2 = df[["SOI" ,"OLR"]]                #Dataframe que incluye SOI y OLR, empieza en 1979 y acaba en abril 2023
df2 = df2.dropna()                      
df3 = df.dropna()                       #Dataframe que incluye todos los datos, empieza en 1982 y acaba en abril 2023

df1.to_csv("SOI.csv")
df2.to_csv("SOI_OLR.csv")
df3.to_csv("Full.csv")

,Nino1.2,Anom1.2,Nino3,Anom3,Nino 4,Anom4,Nino3.4,Anom3.4,OLR,SOI
Date,,,,,,,,,,
1982-01-01,24.28,-0.24,25.84,0.17,28.01,-0.21,26.65,0.08,0.9,1.2
1982-02-01,25.38,-0.72,26.26,-0.11,27.99,-0.11,26.54,-0.20,0.1,0.3
1982-03-01,25.22,-1.38,26.92,-0.25,28.18,-0.05,27.09,-0.14,-0.6,0.6
1982-04-01,24.57,-1.16,27.52,-0.05,28.61,0.10,27.83,0.02,-0.0,0.1
1982-05-01,24.00,-0.62,27.70,0.49,29.19,0.40,28.37,0.49,0.4,-0.3
...,...,...,...,...,...,...,...,...,...,...
2022-12-01,22.54,-0.30,24.48,-0.78,27.71,-0.73,25.81,-0.84,1.7,2.1
2023-01-01,24.27,-0.24,25.17,-0.50,27.62,-0.60,25.88,-0.69,1.8,1.4
2023-02-01,26.80,0.71,26.25,-0.13,27.58,-0.52,26.29,-0.44,1.4,1.4
